In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install pycountry-convert


In [ ]:
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import pandas as pd 
import matplotlib.colors as mcolors 
from matplotlib import cm
import numpy as np
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
import imageio
import pathlib
from geopy.geocoders import Nominatim
import pycountry_convert as pc


In [ ]:
plt.style.use('ggplot')
plt.rcParams['font.family'] = 'sans-serif' 
plt.rcParams['font.serif'] = 'Ubuntu' 
plt.rcParams['font.monospace'] = 'Ubuntu Mono' 
plt.rcParams['font.size'] = 14 
plt.rcParams['axes.labelsize'] = 12 
plt.rcParams['axes.labelweight'] = 'bold' 
plt.rcParams['axes.titlesize'] = 12 
plt.rcParams['xtick.labelsize'] = 12 
plt.rcParams['ytick.labelsize'] = 12 
plt.rcParams['legend.fontsize'] = 12 
plt.rcParams['figure.titlesize'] = 12 
plt.rcParams['image.cmap'] = 'jet' 
plt.rcParams['image.interpolation'] = 'none' 
plt.rcParams['figure.figsize'] = (12, 10) 
plt.rcParams['axes.grid']=True
plt.rcParams['lines.linewidth'] = 2 
plt.rcParams['lines.markersize'] = 8
colors = ['xkcd:pale orange', 'xkcd:sea blue', 'xkcd:pale red', 'xkcd:sage green', 'xkcd:terra cotta', 'xkcd:dull purple', 'xkcd:teal', 'xkcd: goldenrod', 'xkcd:cadet blue',
'xkcd:scarlet']

In [ ]:
data = pd.read_csv('/kaggle/input/manager-dataset/all_coaches.csv')
data.head()

In [ ]:
data[data.current_league=='Premier League']


In [ ]:
data = data.drop(columns=['appointment_date','contract_until_date'])
data = data.dropna()
data = data.drop_duplicates(keep='first')
data.info()


In [ ]:
plt.subplot(2, 1, 1)
sns.countplot(data=data, x='current_league_country', palette='plasma')
plt.xlabel('Country League')
plt.ylabel('Count')

plt.subplot(2, 1, 2)
sns.countplot(data=data, x='current_league', palette='plasma')
plt.xlabel('Leagues')
plt.ylabel('Count')


In [ ]:
geolocator = Nominatim(user_agent="P")
city_data=data.place_of_birth
world_map= folium.Map()
geolocator = Nominatim(user_agent="Piero")
marker_cluster = MarkerCluster().add_to(world_map)
for i in city_data.index.tolist():
    location = geolocator.geocode(city_data.loc[i] + ', ' + data.citizenship.loc[i])
    if location is not None:
        lat, long = location.latitude, location.longitude
        print('Location of city = %s (number %i) found' % (city_data.loc[i], i))
        radius = 5
        folium.CircleMarker(location=[lat, long], radius=radius, fill=True).add_to(marker_cluster)
    else:
        print('Location of city = %s (number %i) not found' % (city_data.loc[i], i))
world_map

In [ ]:
data[data.place_of_birth=='Rosario']

In [ ]:
data[data.place_of_birth=='Santiago']

In [ ]:
data[data.place_of_birth=='Racine, Wisconsin']

In [ ]:
without_club_num = len(data[data.club=='Without Club'])
with_club_num = len(data)-without_club_num
parts = [with_club_num,without_club_num]
colors = ['lightblue','darkorange']
plt.pie(parts,colors=colors,explode = [0.0,0.1],labels=['Coaches with clubs','Coaches without clubs'],autopct='%1.1f%%')

In [ ]:
avg_team_column = np.array(data['avg_term_as_coach'])
avg_team_float = []
for i in range(len(avg_team_column)):
    try:
        avg_team_float.append(float(avg_team_column[i].split(' ')[0]))
    except:
        print(i,'cant be converted')
        continue
avg_team_float = np.array(avg_team_float)

In [ ]:
mean_avg_team = avg_team_float.mean()

In [ ]:
avg_team_float = []
for i in range(len(avg_team_column)):
    try:
        avg_team_float.append(float(avg_team_column[i].split(' ')[0]))
    except:
        avg_team_float.append(mean_avg_team)
        continue

In [ ]:
avg_team_float = np.array(avg_team_float)
sns.histplot(avg_team_float, bins=20, kde=True, color="green")
plt.title("Distribution of Average Term as Coach (in Years)")
plt.xlabel('Average Term as a Coach')
plt.ylabel("Number of Coaches")
plt.show()

In [ ]:
coaches_country = data.current_league_country.unique()
data['avg_term_as_coach'] = avg_team_float
y = []
x = []
j = 1

for country in coaches_country:
    country_data = data[data.current_league_country == country]
    if not country_data.empty:  # Check if the filtered DataFrame is not empty
        y.append(np.array(country_data['avg_term_as_coach']))
        x.append([j] * len(country_data))
        j = j + 1

color = ['red','gold','lightblue','darkorange','green']
plt.figure(figsize=(11,11))
i = 0

for country in coaches_country:
    plt.subplot(5,1,i+1)
    country_data = data[data.current_league_country == country]
    plt.title('Coaches Country = %s' % (country), fontsize=20)
    
    # Use the x argument to specify the data for the boxplot
    sns.boxplot(x=country_data['avg_term_as_coach'], color=color[i])
    
    plt.xlim(0, avg_team_float.max() + 1)
    i = i + 1

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(11,11))
i = 0
for country in coaches_country:
    plt.subplot(5,1,i+1)
    country_data = data[data.current_league_country == country]
    plt.title('Coaches Country = %s' % (country), fontsize=25, fontweight='bold')
    
    # Explicitly set the data for the x-axis
    sns.boxplot(x=country_data['avg_term_as_coach'], color=color[i])
    
    plt.xlim(0,5)
    plt.xlabel('Average Length of Contract', fontsize=20)
    i = i + 1

plt.tight_layout()
plt.show()


In [ ]:
country_describe = []
country_columns = data.avg_term_as_coach.describe().index
for country in coaches_country:
    country_data = data[data.current_league_country==country]
    country_describe.append(country_data.avg_term_as_coach.describe().values)
data_summary = pd.DataFrame(country_describe,columns = country_columns).T
data_summary.columns = coaches_country
data_summary.drop('count').plot(color = color)
plt.xlabel('Average Term Statistics')

In [ ]:
data['preffered_formation']=data['preffered_formation'].replace('1.46 Years','4-2-3-1')
plt.figure(figsize=(7, 5))
sns.barplot(x=data['preffered_formation'].value_counts().index, y=data['preffered_formation'].value_counts(),palette="viridis")
plt.title('Preferred Formation Count Plot', fontsize=30, fontweight='bold')
plt.ylabel('Numberof Coaches')
plt.xlabel('Formation')
plt.xticks(rotation=45)


In [ ]:
all_formation = data['preffered_formation'].value_counts().index
formation_country = []
for country in coaches_country:
    country_data = data[data.current_league_country==country]
    country_data = country_data.reset_index()
    formation_tot = []
    for formation in all_formation:
        formation_count = 0
        for i in range(len(country_data)):
            if country_data['preffered_formation'].loc[i]==formation:
                formation_count = formation_count + 1
        formation_tot.append(formation_count)
    formation_country.append(formation_tot)
plt.figure(figsize=(20, 20))
plt.subplot(6, 1, 1)
sns.barplot(x=data['preffered_formation'].value_counts().index, y=data['preffered_formation'].value_counts())
plt.title('Preferred Formation Count Plot', fontsize=30, fontweight='bold')
plt.ylabel('Count')
plt.xlabel('Formation')

plt.xticks(rotation=45)

i = 1
leagues = data.current_league.unique()
for league in leagues:
    plt.subplot(6, 1, i + 1)
    sns.barplot(x=all_formation, y=formation_country[i - 1])
    plt.title('Preferred Formation Count Plot of %s' % league, fontsize=30, fontweight='bold')
    plt.ylabel('Count')
    plt.xlabel('Formation')
    

    i = i + 1

plt.tight_layout()

In [ ]:
# Plotting the age distribution of coaches
plt.figure(figsize=(10, 6))
sns.distplot(data.age, bins=20, color='blue')
plt.xlabel('Age of Coaches')
plt.ylabel('Density')
plt.title('Age Distribution of Coaches')
plt.show()


In [ ]:
# Plotting age distribution of coaches by country
plt.figure(figsize=(20, 20))
for idx, country in enumerate(coaches_country):
    plt.subplot(5, 1, idx + 1)
    sns.distplot(data[data.current_league_country == country]['age'], bins=20, color=color[idx])
    plt.xlabel('Age of Coaches')
    plt.ylabel('Density')
    plt.title(f'Age Distribution of Coaches in {country}')

plt.tight_layout()
plt.show()

In [ ]:
# Calculate age statistics by country
country_describe = []
country_columns = data.age.describe().index
for country in coaches_country:
    country_data = data[data.current_league_country == country]
    country_describe.append(country_data.age.describe().values)

# Create a summary dataframe and plot
data_summary_age = pd.DataFrame(country_describe, columns=country_columns).T
data_summary_age.columns = coaches_country

# Dropping the 'count' and 'std' rows for a cleaner visualization
data_summary_age.drop(['count', 'std']).plot(color=color, figsize=(15, 7))
plt.title('Age Statistics of Coaches by Country')
plt.ylabel('Age')
plt.xlabel('Statistical Measure')
plt.legend(title='Country')
plt.show()

# Return the summary dataframe for further insights
data_summary_age


In [ ]:
import pandas as pd
# Loading the newly uploaded files
all_coaches_new = pd.read_csv("/kaggle/input/manager-dataset/all_coaches.csv", encoding='ISO-8859-1')
all_trophies_new = pd.read_csv("/kaggle/input/manager-dataset/all_trophies.csv", encoding='ISO-8859-1')

# Merging the datasets
merged_data_new = pd.merge(all_coaches_new, all_trophies_new, on='name', how='inner')

merged_data_new.info()


In [ ]:
# Try reading the datasets using a different encoding
try:
    trophies_df = pd.read_csv('/kaggle/input/manager-dataset/all_trophies.csv', encoding='ISO-8859-1')
    coaches_df = pd.read_csv('/kaggle/input/manager-dataset/all_coaches.csv', encoding='ISO-8859-1')
    
    # Display the first few rows of each dataset
    trophies_df_head = trophies_df.head()
    coaches_df_head = coaches_df.head()
except Exception as e:
    error_message = str(e)
    trophies_df_head, coaches_df_head = None, None

trophies_df_head, coaches_df_head, error_message if "error_message" in locals() else None


In [ ]:
# Check for missing values in both datasets
trophies_missing = trophies_df.isnull().sum()
coaches_missing = coaches_df.isnull().sum()

# Check data types
trophies_data_types = trophies_df.dtypes
coaches_data_types = coaches_df.dtypes

trophies_missing, coaches_missing, trophies_data_types, coaches_data_types


In [ ]:
# Re-define the date columns list
date_columns = ['birth_date', 'appointment_date', 'contract_until_date']

# Convert date columns to datetime format
for col in date_columns:
    coaches_df[col] = pd.to_datetime(coaches_df[col], errors='coerce')

# Display the first few rows of the processed coaches dataset
coaches_df_head_processed = coaches_df.head()

coaches_df_head_processed


In [ ]:
# Merge the two datasets on the 'name' column
merged_df = pd.merge(coaches_df, trophies_df, on='name', how='inner')

# Display the first few rows of the merged dataset
merged_df_head = merged_df.head()

merged_df_head


In [ ]:
# Create a new column 'total_trophies' which is the sum of all the trophy columns
trophy_columns = trophies_df.columns.drop('name')
merged_df['total_trophies'] = merged_df[trophy_columns].sum(axis=1)

# Group by 'preffered_formation' and calculate the average number of trophies won
formation_vs_trophies = merged_df.groupby('preffered_formation')['total_trophies'].mean().sort_values(ascending=False)

formation_vs_trophies


In [ ]:
merged_df.info()

In [ ]:
# Group by 'avg_term_as_coach' and calculate the average number of trophies won
term_vs_trophies = merged_df.groupby('avg_term_as_coach')['total_trophies'].mean().sort_values(ascending=False)

term_vs_trophies


In [ ]:
# Replace '1.46 Years' with '4-2-3-1' in the 'preffered_formation' column
merged_df['preffered_formation'] = merged_df['preffered_formation'].replace('1.46 Years', '4-2-3-1')

# Replace 'UEFA Pro Licence' with 1.46 in the 'avg_term_as_coach' column
merged_df['avg_term_as_coach'] = merged_df['avg_term_as_coach'].replace({'UEFA Pro Licence': 1.46})

# Drop duplicate rows based on the 'name' column
merged_df = merged_df.drop_duplicates(subset='name', keep='first')

# Re-run the analyses

# 1. Preferred Formation vs Trophies Won
formation_vs_trophies_updated = merged_df.groupby('preffered_formation')['total_trophies'].mean().sort_values(ascending=False)

# 2. Average Term as Coach vs Trophies Won
term_vs_trophies_updated = merged_df.groupby('avg_term_as_coach')['total_trophies'].mean().sort_values(ascending=False)

